# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Load csvs as dfs

In [8]:
robados = pd.read_csv('./curados/robados.csv', parse_dates= ['tramite_fecha', 'fecha_inscripcion_inicial'])
recuperados = pd.read_csv('./curados/recuperados.csv', parse_dates= ['tramite_fecha', 'fecha_inscripcion_inicial'])

In [10]:
#recuperados.dtypes

# Start exploring

In [ ]:
# para robados['dia_semana'] 0 = Lunes a 6 = Domingo